In [11]:
import pandas as pd
import numpy as np
import re

#Causality
Causal_scores = pd.read_csv("/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Merged_Best_causality.Table")

#gwas STRs (in literature)
STR_gwas_list = ['JARID2','TH','ATXN1','GCLC','NUMBL','PHOX2B','CCK','CETP','CNBP','CYP19A1','GC','GRIK1','LEP','MIF','MOG','PTHR1','TCF7L2','TFAP2B','TNFRSF8','AFF2','AR','ARX','ASCL1','ATN1','ATXN2','ATXN3','ATXN7','ATXN8OS','CACNA1A','CBL','CNDP1','CNR1','DIP2B','DMPK','FMR1','FOXC1','FRA10A','FXN','GCLC','GSPT1','HD','HOXA13','HOXD13','JPH3','KCNN3','MECP2B','MICA','NCOA3','NOS1','PABPN1','PAX7','PCSK9','PHOX2B','POLG','PPP2R2B','RENL','RETN','RUNX2','SIX5','TBP','TGFBR1','VLDLR']
#gwas file -- downloaded from 
gwas = pd.read_csv("/storage/szfeupe/Runs/650GTEx_estr/gwas/gwas-catalog-associations.tsv", sep='\t') 
gwas.columns
######
selectedcolumns=[ 'DISEASE/TRAIT',  'REGION', 'CHR_ID', 'CHR_POS','REPORTED GENE(S)', 'MAPPED_GENE', 'SNP_GENE_IDS', 'UPSTREAM_GENE_DISTANCE', 'DOWNSTREAM_GENE_DISTANCE', 'STRONGEST SNP-RISK ALLELE', 'SNPS',  'CONTEXT', 'INTERGENIC', 'RISK ALLELE FREQUENCY', 'P-VALUE', 'PVALUE_MLOG', 'OR or BETA']
for i in set(list(gwas['DISEASE/TRAIT'])):
    if list(gwas['DISEASE/TRAIT']).count(i)>300:
        print(i, list(gwas['DISEASE/TRAIT']).count(i))
        
selectedcolumns = ['DISEASE/TRAIT',  'REGION', 'CHR_ID', 'CHR_POS','REPORTED GENE(S)', 'MAPPED_GENE','SNP_GENE_IDS', 'SNPS',  'CONTEXT', 'INTERGENIC', 'RISK ALLELE FREQUENCY', 'P-VALUE', 'PVALUE_MLOG' ]

/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (12,23,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Body mass index 841
Schizophrenia 899
Type 2 diabetes 396
Inflammatory bowel disease 319
Rheumatoid arthritis 309
Obesity-related traits 957
Height 823
Post bronchodilator FEV1/FVC ratio in COPD 473
IgG glycosylation 699
Post bronchodilator FEV1 1182
Post bronchodilator FEV1/FVC ratio 1631
Crohn's disease 390


In [12]:
#Get genes associated with trait from gwas studies (SNPs)

trait='Schizophrenia'
condition = gwas.loc[gwas['DISEASE/TRAIT']==trait][selectedcolumns].sort_values("CHR_ID")
genes = []
for g in [x.split('-') for x in list(condition['MAPPED_GENE'].astype(str))]:
    genes=genes+g
print(len(genes))
genes = [x.strip() for x in genes if 'LOC' not in x]
print(len(genes))
condition['MAPPED_GENE']=condition['MAPPED_GENE'].astype(str).apply(lambda x: x.split('-')+['*'])

1312
946


In [13]:
#Exploring STRs in GTEx
merged_t = pd.read_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/LR_SummaryTest_Table.tsv', sep='\t')
summary = pd.read_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Nerve-Tibial/Master.table', sep='\t')
names = summary[['gene', 'gene.name']].copy().drop_duplicates()
print(summary.shape)
gwas_s=genes
print(len(gwas_s))
gwas_genes = list(set(list(summary.loc[summary['gene.name'].isin(gwas_s)]['gene'])))
print(len(gwas_genes))
numb_tests = merged_t.loc[merged_t['gene'].isin(gwas_genes)]
print('Number of tests ', numb_tests.shape[0])
gwas_egenes = numb_tests.loc[numb_tests['E.tissues']>=1]
print('Number of gwas eGenes for schizophrenia: ',len(set(list(gwas_egenes['gene']))))
Finalset = gwas_egenes[['gene','chrom','str.start','E.tissues','str.end','period','motif']].merge(names)
Finalset['str.len']=Finalset['str.end'].astype(int)-Finalset['str.start'].astype(int)
#In gwas
condition['in.estr']=condition['MAPPED_GENE'].apply(lambda x: 1 if (x[0] in genes) or (x[1] in genes) else 0)
Finalset

(88752, 29)
946
430
Number of tests  3634
Number of gwas eGenes for schizophrenia:  190


,gene,chrom,str.start,E.tissues,str.end,period,motif,gene.name,str.len
0,ENSG00000177674.11,chr1,11793667,1.0,11793684,1,A,AGTRAP,17
1,ENSG00000054282.11,chr1,243394402,12.0,243394412,1,A,SDCCAG8,10
2,ENSG00000121766.10,chr1,31672838,1.0,31672886,2,AC,ZCCHC17,48
3,ENSG00000121766.10,chr1,31685250,1.0,31685267,1,A,ZCCHC17,17
4,ENSG00000142949.12,chr1,44025261,1.0,44025272,2,AC,PTPRF,11
5,ENSG00000120341.14,chr1,177982159,1.0,177982169,1,A,SEC16B,10
6,ENSG00000120341.14,chr1,178037855,1.0,178037909,2,AC,SEC16B,54
7,ENSG00000120341.14,chr1,177918519,1.0,177918565,2,AC,SEC16B,46
8,ENSG00000079277.15,chr1,47014814,1.0,47014832,1,A,MKNK1,18
9,ENSG00000079277.15,chr1,47071277,3.0,47071296,1,A,MKNK1,19


In [22]:
newselct =condition.loc[condition['in.estr']==1]
print(newselct.shape)
newselct = newselct.dropna(subset=['CHR_ID'])
newselct['chrom']='chr'+newselct['CHR_ID'].astype(str)
newselct['snp.start']=newselct['CHR_POS'].astype(int)
newselct['gene.name']=newselct['MAPPED_GENE'].apply(lambda x: str(x[0]))
data = Finalset.merge(newselct[['chrom', 'snp.start','gene.name','SNPS', "CONTEXT"]],on=['chrom','gene.name'])
print(data.shape)
data = newselct[['chrom', 'snp.start','gene.name','SNPS', "CONTEXT"]].merge(Finalset, on=['chrom','gene.name'])
print(data.shape)
data

(502, 14)
(246, 12)
(246, 12)


,chrom,snp.start,gene.name,SNPS,CONTEXT,gene,str.start,E.tissues,str.end,period,motif,str.len
0,chr1,35818353,AGO4,rs11264213,intron_variant,ENSG00000134698.10,36270731,3.0,36270758,1,A,27
1,chr1,35818353,AGO4,rs11264213,intron_variant,ENSG00000134698.10,36284227,1.0,36284305,4,AAGG,78
2,chr1,243500591,AKT3,rs14403,3_prime_UTR_variant,ENSG00000117020.12,243861275,1.0,243861289,1,A,14
3,chr1,243527189,AKT3,rs13376709,intron_variant,ENSG00000117020.12,243861275,1.0,243861289,1,A,14
4,chr1,155906822,RIT1,rs3856261,intron_variant,ENSG00000143622.6,155915072,4.0,155915114,2,AC,42
5,chr1,243286579,SDCCAG8,rs3818802,intron_variant,ENSG00000054282.11,243394402,12.0,243394412,1,A,10
6,chr1,243381525,SDCCAG8,rs1538774,intron_variant,ENSG00000054282.11,243394402,12.0,243394412,1,A,10
7,chr1,243445665,SDCCAG8,rs6703335,intron_variant,ENSG00000054282.11,243394402,12.0,243394412,1,A,10
8,chr1,243391803,SDCCAG8,rs77149735,intron_variant,ENSG00000054282.11,243394402,12.0,243394412,1,A,10
9,chr1,243391803,SDCCAG8,rs77149735,intron_variant,ENSG00000054282.11,243394402,12.0,243394412,1,A,10


In [27]:
data['distance']=data['str.start'] - data['snp.start']
data['within_100k'] = data['distance'].apply(lambda x: 1 if abs(x)<100000 else 0)
print(data.shape)
print('There are %s eGenes with associates eSNPs for schizophrenia '%len(set(list(data['gene']))))
data.loc[data['within_100k']==1]

(246, 14)
There are 134 eGenes with associates eSNPs for schizophrenia 


,chrom,snp.start,gene.name,SNPS,CONTEXT,gene,str.start,E.tissues,str.end,period,motif,str.len,distance,within_100k
4,chr1,155906822,RIT1,rs3856261,intron_variant,ENSG00000143622.6,155915072,4.0,155915114,2,AC,42,8250,1
6,chr1,243381525,SDCCAG8,rs1538774,intron_variant,ENSG00000054282.11,243394402,12.0,243394412,1,A,10,12877,1
7,chr1,243445665,SDCCAG8,rs6703335,intron_variant,ENSG00000054282.11,243394402,12.0,243394412,1,A,10,-51263,1
8,chr1,243391803,SDCCAG8,rs77149735,intron_variant,ENSG00000054282.11,243394402,12.0,243394412,1,A,10,2599,1
9,chr1,243391803,SDCCAG8,rs77149735,intron_variant,ENSG00000054282.11,243394402,12.0,243394412,1,A,10,2599,1
10,chr1,8329994,SLC45A1,rs12142465,intron_variant,ENSG00000162426.10,8373565,1.0,8373584,1,A,19,43571,1
15,chr1,204630167,LRRN2,rs7551222,intron_variant,ENSG00000170382.7,204564282,1.0,204564326,3,AAT,44,-65885,1
21,chr1,155017821,ZBTB7B,rs2242194,3_prime_UTR_variant,ENSG00000160685.9,155027695,1.0,155027718,3,AAT,23,9874,1
22,chr1,150567705,ADAMTSL4,rs72700829,intron_variant,ENSG00000143382.9,150552235,1.0,150552290,6,CCCCGG,55,-15470,1
23,chr1,205066327,CNTN2,rs16937,intron_variant,ENSG00000184144.5,205049654,1.0,205049690,2,AC,36,-16673,1


In [39]:
Causal_scores = pd.read_csv("/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Merged_Best_causality.Table", sep='\t')
All_gwas = Causal_scores.loc[Causal_scores['gene'].isin(list(set(list(data['gene']))))]
All_gwas.loc[All_gwas['best.q']<=.1]

,gene,chrom,best.str.start,best.score,best.q,best.tissue,NumTissues,gene.name,top.variant,best.llq
9,ENSG00000172137.14,chr16,71335226.0,4.545600e-03,0.057028,Heart-LeftVentricle,17,CALB2,SNP_71308106,0.065267
229,ENSG00000100731.11,chr14,71416109.0,4.541180e-03,0.002441,Esophagus-Mucosa,17,PCNX,SNP_71401112,0.001157
331,ENSG00000182667.10,chr11,131925517.0,1.389730e-02,0.001758,Nerve-Tibial,17,NTM,SNP_131991325,0.000833
524,ENSG00000100207.14,chr22,42819557.0,2.421520e-01,0.030173,Nerve-Tibial,17,TCF20,STR_42819557,0.008556
1184,ENSG00000140836.10,chr16,73089578.0,4.615650e-02,0.091874,Esophagus-Mucosa,17,ZFHX3,SNP_73089716,0.057791
1276,ENSG00000153707.11,chr9,9595947.0,6.680190e-02,0.011214,Nerve-Tibial,17,PTPRD,STR_9595947,0.001006
1281,ENSG00000151422.8,chr5,108032282.0,5.620240e-02,0.092540,Muscle-Skeletal,17,FER,STR_108032282,0.021767
1331,ENSG00000134698.10,chr1,36270731.0,8.211240e-02,0.087322,Artery-Tibial,17,AGO4,STR_36270731,0.057141
1770,ENSG00000166444.13,chr11,8839810.0,4.352600e-02,0.086315,Adipose-Subcutaneous,17,ST5,STR_8839810,0.088472
1822,ENSG00000070366.9,chr17,2129570.0,5.552770e-02,0.090078,Cells-Transformedfibroblasts,17,SMG6,SNP_2026417,0.043943


In [35]:
#STR gwas hits
print(len(STR_gwas_list), len(set(STR_gwas_list)))
merged_t = merged_t.merge(names)
Hits_gtex = merged_t.loc[merged_t['gene.name'].isin(STR_gwas_list)]
Hits_gtex.loc[Hits_gtex['E.tissues']>=1].sort_values('gene.name')

62 60


,chrom,str.start,str.id,gene,E.tissues,ll.E.tissues,eSTR.Adipose.S,eSTR.Adipose.S.ll,qval.Adipose.S,llqval.Adipose.S,...,beta.Thyroid,eSTR.Blood,eSTR.Blood.ll,qval.Blood,llqval.Blood,beta.Blood,str.end,period,motif,gene.name
10377,chr12,7034264,STR_7034264,ENSG00000111676.10,8.0,8.0,1.0,1.0,0.002085,0.001238,...,-0.401075,0.0,0.0,1.000000,0.811180,-0.027276,7034322,2,AC,ATN1
14660,chr14,92557796,STR_92557796,ENSG00000066427.17,10.0,9.0,1.0,1.0,0.002077,0.001046,...,0.260183,1.0,0.0,0.098919,0.159107,-0.154160,92557821,4,AAAG,ATXN3
32869,chr19,13729541,STR_13729541,ENSG00000141837.14,1.0,1.0,0.0,0.0,0.000000,0.812012,...,0.135937,0.0,0.0,NaN,0.851923,-0.005792,13729573,2,AG,CACNA1A
7587,chr11,119087050,STR_119087050,ENSG00000110395.4,2.0,2.0,0.0,0.0,1.000000,0.788202,...,-0.064029,0.0,0.0,NaN,0.721164,-0.054110,119087097,4,AAAT,CBL
28256,chr18,72210730,STR_72210730,ENSG00000150656.10,1.0,1.0,0.0,0.0,0.000000,0.342899,...,-0.056906,0.0,0.0,NaN,0.851343,-0.005813,72210778,5,AAAAT,CNDP1
16802,chr15,51620745,STR_51620745,ENSG00000137869.9,1.0,1.0,0.0,0.0,1.000000,0.843324,...,-0.025783,0.0,0.0,1.000000,0.808066,0.028981,51620782,2,AC,CYP19A1
11100,chr12,51123498,STR_51123498,ENSG00000066084.8,1.0,1.0,0.0,0.0,0.000000,0.619114,...,0.360361,0.0,0.0,0.635046,0.329699,0.120274,51123531,1,A,DIP2B
33138,chr19,46265048,STR_46265048,ENSG00000104936.13,1.0,4.0,0.0,0.0,0.000000,0.776608,...,0.290464,0.0,0.0,NaN,0.292570,0.122120,46265085,6,AGCTCC,DMPK
33140,chr19,46308960,STR_46308960,ENSG00000104936.13,1.0,3.0,0.0,0.0,0.000000,0.260466,...,0.064932,0.0,0.0,NaN,0.836187,-0.014291,46308980,4,AAAG,DMPK
91616,chr9,71685708,STR_71685708,ENSG00000165060.7,1.0,1.0,0.0,0.0,0.000000,0.706640,...,0.088532,0.0,0.0,0.736591,0.449685,0.105619,71685728,1,A,FXN
